In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn
%matplotlib inline   
                    #可以使绘图直接显示在代码之中，相当于plt.show

In [2]:
data=pd.read_csv('HR.csv')

In [3]:
data

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,part,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2,151,3,0,1,0,support,low
14995,0.37,0.48,2,160,3,0,1,0,support,low
14996,0.37,0.53,2,143,3,0,1,0,support,low
14997,0.11,0.96,6,280,4,0,1,0,support,low


In [4]:
data=data.join(pd.get_dummies(data['salary']))

In [5]:
del data['salary']

In [6]:
data=data.join(pd.get_dummies(data['part']))

In [7]:
del data['part']

In [8]:
data

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,high,low,...,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical
0,0.38,0.53,2,157,3,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.72,0.87,5,223,5,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0.37,0.52,2,159,3,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2,151,3,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
14995,0.37,0.48,2,160,3,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
14996,0.37,0.53,2,143,3,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
14997,0.11,0.96,6,280,4,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [9]:
data.left.value_counts()

0    11428
1     3571
Name: left, dtype: int64

In [10]:
data.left.mean()

0.2380825388359224

In [11]:
len(data)

14999

In [12]:
11428/len(data)   #标准预测正确率（全预测不离职的正确率），最后模型一定要比这个高

0.7619174611640777

In [50]:
Y_data=data.left.values.reshape(-1,1)

In [51]:
Y=torch.from_numpy(Y_data).type(torch.float32)


In [14]:
[c for c in data.columns if c!='left']

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'promotion_last_5years',
 'high',
 'low',
 'medium',
 'IT',
 'RandD',
 'accounting',
 'hr',
 'management',
 'marketing',
 'product_mng',
 'sales',
 'support',
 'technical']

In [15]:
X_data=data[[c for c in data.columns if c!='left']].values

In [64]:
X=torch.from_numpy(X_data).float()

In [66]:
X.dtype

torch.float32

In [17]:
X.size()

torch.Size([14999, 20])

In [18]:
Y.shape

torch.Size([14999, 1])

# 创建模型

In [19]:
class Model(nn.Module):
    def __init__(self):                  #初始化这些层
        super().__init__()
        self.linear_1=nn.Linear(20,64)   #多层感知器，创建64个中间层（隐藏层）
        self.linear_2=nn.Linear(64,64)
        self.linear_3=nn.Linear(64,1)
        self.relu=nn.ReLU()              #一个relu激活层
        self.sigmoid=nn.Sigmoid()        #因为是二分类问题，所以再创建一个sigmoid层
    def forward(self,input):             #在forward中使用这些层，对input进行处理
        x=self.linear_1(input)           #利用输入调用第一层得到x
        x=self.relu(x)                   #对第一层使用激活函数
        x=self.linear_2(x)               #调用第二层
        x=self.relu(x)
        x=self.linear_3(x)               #调用第三层
        x=self.sigmoid(x)                #最后一层用sigmoid激活函数
        return(x)

In [20]:
model=Model()

In [21]:
model

Model(
  (linear_1): Linear(in_features=20, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=64, bias=True)
  (linear_3): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [22]:
import torch.nn.functional as F

# 改写模型

In [23]:
class Model(nn.Module):
    def __init__(self):                  
        super().__init__()
        self.linear_1=nn.Linear(20,64)   
        self.linear_2=nn.Linear(64,64)
        self.linear_3=nn.Linear(64,1)
         # self.relu=nn.ReLU()              
         # self.sigmoid=nn.Sigmoid()        用F的方法就没有必要调用这两层了
        
    def forward(self,input):             
        x=F.relu(self.linear_1(input))    #利用F.relu函数直接对Linear激活
          #x=self.relu(x)                   #就不用再写激活函数了
        x=F.relu(self.linear_2(x))              
          #x=self.relu(x)
        x=torch.sigmoid(self.linear_3(x))     #可以看出这样就简洁了许多           
          #x=self.sigmoid(x)                    #(改成torch.sigmoid不报错)
        return(x)

In [24]:
model=Model()

In [25]:
model

Model(
  (linear_1): Linear(in_features=20, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=64, bias=True)
  (linear_3): Linear(in_features=64, out_features=1, bias=True)
)

In [26]:
def get_model():       #编写一个函数，每次运行这个函数都得到这个模型以及优化方法
    model=Model()
    opt=torch.optim.Adam(model.parameters(),lr=0.001)
    return model,opt

In [27]:
model,optim=get_model()

# 定义损失函数

In [28]:
loss_fn=nn.BCELoss()

In [29]:
batches=64
no_of_batches=len(data)//batches
epoches=100

In [30]:
for epoch in range(epoches):
    for batch in range(no_of_batches):
        start=batch*batches
        end=start+batches
        x=X[start:end]
        y=Y[start:end]
        y_pred=model(x)
        loss=loss_fn(y_pred,y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    #with torch.no_grad():
        #print('epoch: ',epoch,'loss: ',loss_fn(model(X),Y).data.item())

In [31]:
loss_fn(model(X),Y)

tensor(0.5491, grad_fn=<BinaryCrossEntropyBackward>)

In [32]:
loss_fn(model(X),Y).data

tensor(0.5491)

In [33]:
loss_fn(model(X),Y).data.item()

0.5490880012512207

In [34]:
((model(X).data.numpy()>0.5)==Y.data.numpy()).mean()

0.7619174611640777

# 使用dataset类进行重构

In [35]:
from torch.utils.data import TensorDataset    #untils里有很多实用的小工具

In [36]:
HRdataset=TensorDataset(X,Y)

In [38]:
HRdataset[2:5]

(tensor([[1.1000e-01, 8.8000e-01, 7.0000e+00, 2.7200e+02, 4.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
          0.0000e+00, 0.0000e+00],
         [7.2000e-01, 8.7000e-01, 5.0000e+00, 2.2300e+02, 5.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
          0.0000e+00, 0.0000e+00],
         [3.7000e-01, 5.2000e-01, 2.0000e+00, 1.5900e+02, 3.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
          0.0000e+00, 0.0000e+00]]),
 tensor([[1.],
         [1.],
         [1.]]))

In [40]:
for epoch in range(epoches):
    for batch in range(no_of_batches):
        x,y=HRdataset[batch*batches:batch*batches+batches]
            #start=batch*batches
            #end=start+batches
            #x=X[start:end]
            #y=Y[start:end]
        y_pred=model(x)
        loss=loss_fn(y_pred,y)
        optim.zero_grad()
        loss.backward()
        optim.step()

# dataload类

In [42]:
from torch.utils.data import DataLoader

In [43]:
HR_ds=TensorDataset(X,Y)
HR_dl=DataLoader(HR_ds,batch_size=batch,shuffle=True) #第一个参数是一个dataset集，
                                #另外两个常用的参数分别是切片的大小和是否乱序

In [44]:
model,optim=get_model()

In [45]:
for epoch in range(epoches):
    for x,y in HR_dl:
        #for batch in range(no_of_batches):
            #x,y=HRdataset[batch*batches:batch*batches+batches]
        y_pred=model(x)
        loss=loss_fn(y_pred,y)
        optim.zero_grad()
        loss.backward()
        optim.step()

# 添加验证

了解过拟合与欠拟合

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
train_x,text_x,train_y,text_y=train_test_split(X_data,Y_data)
                                            #默认25%的数据被划分为测试数据

In [59]:
X_data.shape

(14999, 20)

In [60]:
train_x.shape,text_x.shape

((11249, 20), (3750, 20))

In [61]:
train_x=torch.from_numpy(train_x).type(torch.float32)
train_y=torch.from_numpy(train_y).type(torch.float32)
text_x=torch.from_numpy(text_x).type(torch.float32)
text_y=torch.from_numpy(text_y).type(torch.float32)

In [63]:
train_ds=TensorDataset(train_x,train_y)
train_dl=DataLoader(train_ds,batch_size=batches,shuffle=True)
text_ds=TensorDataset(text_x,text_y)
text_dl=DataLoader(text_ds,batch_size=batches)#测试数据没有必要乱序

如何计算正确率

In [67]:
def accuracy(y_pred,y_true):              #定义一个计算正确率的函数
    y_pred=(y_pred>0.5).type(torch.int32) #先把预测值转换为0和1
    acc=(y_pred==y_true).float().mean()   #预测值与真实值进行比较,.
                        #float是快捷的转换格式的方法，再求平均值就是平均正确率了
    return acc

In [73]:
model,optim=get_model()    #重新初始化一下模型

In [78]:
for epoch in range(epoches):
    for x,y in train_dl:
        y_pred=model(x)
        loss=loss_fn(y_pred,y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    with torch.no_grad():
        epoch_accuracy=accuracy(model(train_x),train_y)  #每把数据训练一遍就计算一遍正确率
        epoch_loss=loss_fn(model(train_x),train_y).data
        epoch_text_accuracy=accuracy(model(text_x),text_y)  #计算测试数据的正确率
        epoch_text_loss=loss_fn(model(text_x),text_y).data
        print('epoch: ',epoch,'loss: ',round(epoch_loss.item(),3),
                              'accuracy: ',round(epoch_accuracy.item(),3),
                              'text_loss: ',round(epoch_text_loss.item(),3),
                              'text_accuracy: ',round(epoch_text_accuracy.item(),3))
                       #打印结果有点长，可以使用round(a,3)的方法让a保留小数点后三位

epoch:  0 loss:  0.566 accuracy:  0.759 text_loss:  0.557 text_accuracy:  0.772
epoch:  1 loss:  0.56 accuracy:  0.759 text_loss:  0.539 text_accuracy:  0.772
epoch:  2 loss:  0.505 accuracy:  0.759 text_loss:  0.49 text_accuracy:  0.772
epoch:  3 loss:  0.475 accuracy:  0.769 text_loss:  0.469 text_accuracy:  0.777
epoch:  4 loss:  0.438 accuracy:  0.754 text_loss:  0.425 text_accuracy:  0.766
epoch:  5 loss:  0.421 accuracy:  0.846 text_loss:  0.419 text_accuracy:  0.846
epoch:  6 loss:  0.386 accuracy:  0.77 text_loss:  0.372 text_accuracy:  0.779
epoch:  7 loss:  0.403 accuracy:  0.753 text_loss:  0.383 text_accuracy:  0.765
epoch:  8 loss:  0.321 accuracy:  0.881 text_loss:  0.314 text_accuracy:  0.882
epoch:  9 loss:  0.31 accuracy:  0.869 text_loss:  0.298 text_accuracy:  0.871
epoch:  10 loss:  0.328 accuracy:  0.873 text_loss:  0.326 text_accuracy:  0.867
epoch:  11 loss:  0.303 accuracy:  0.862 text_loss:  0.288 text_accuracy:  0.862


KeyboardInterrupt: 